In [1]:
from parameters import FileParams as fp
import os
import numpy as np

In [2]:
"""
Load training data
"""
train_x = np.load(os.path.join(fp.data_dir, 'train_x.npy'))
train_y = np.load(os.path.join(fp.data_dir, 'train_y.npy'))
print(f'Percent of data positive: {np.mean(train_y)}')
print('train_x size: ', train_x.nbytes / 1024 / 1024, 'MB')
print(len(train_x))

Percent of data positive: 0.19020399013674064
train_x size:  1975.988296508789 MB
686994


In [3]:
"""
Load validation data
"""
val_x = np.load(os.path.join(fp.data_dir, 'val_x.npy'))
val_y = np.load(os.path.join(fp.data_dir, 'val_y.npy'))
print(f'Percent of data positive: {np.mean(val_y)}')
print('val_y size: ', val_x.nbytes / 1024 / 1024, 'MB')

Percent of data positive: 0.00025210790218213396
val_y size:  102.68038177490234 MB


In [5]:
from keras.callbacks import TensorBoard, ModelCheckpoint


BATCH_SIZE = 128
EPOCHS = 4


def create_and_train_model(model_name: str, model_funct, funct_args, batch_size=BATCH_SIZE, epochs=EPOCHS):
    model_checkpoint_callback = ModelCheckpoint(
        filepath=f'./checkpoints/{model_name}',
        save_weights_only=False,  # Save the whole model
        monitor='val_loss',
        mode='min',
        save_best_only=True)

    log_dir = os.path.join(fp.log_dir, model_name)

    my_model = model_funct(*funct_args)
    my_model.summary()
    my_model.fit(
        train_x,
        train_y,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(val_x, val_y),
        callbacks=[TensorBoard(log_dir), model_checkpoint_callback]
    )

In [4]:
"""
Test different sizes of GRU units
"""
from create_models import variable_GRU_units_model


units = [4, 8, 10, 16, 20, 24]
for unit in units:
    print(f'\n\nGRU units: {unit}')
    create_and_train_model(f'model_GRU_{unit}', variable_GRU_units_model, (unit,))

"""
After testing, seems like GRU units of around 16 or 20 is best
"""



GRU units: 4
Model: "model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4)                 228       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 233
Trainable params: 233
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
5368/5368 [==============================] - 164s 30ms/step - loss: 0.1426 - accuracy: 0.9405 - val_loss: 0.0316 - val_accuracy: 0.9924
Epoch 2/4
5368/5368 [==============================] - 159s 30ms/step - loss: 0.0383 - accuracy: 0.9865 - val_loss: 0.0168 - val_accuracy: 0.9954
Epoch 3/4
5368/5368 [==============================] - 159s 30ms/step - loss: 0.0303 - accuracy: 0.9893 - val_loss: 0.0113 - val_accuracy: 0.9964
Epoch

In [10]:
"""
Test different intermediate layer sizes
"""
import create_models
import importlib
importlib.reload(create_models)

layers = [5, 8, 10]
for layer in layers:
    print(f'\n\nIntermediate layers: {layer}')
    create_and_train_model(f'model_GRU_16_layers_{layer}', create_models.variable_intermediate_layer_model, (layer,), epochs=6)



Intermediate layers: 5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_8 (GRU)                 (None, 16)                1488      
                                                                 
 dense_10 (Dense)            (None, 5)                 85        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 1,579
Trainable params: 1,579
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
5368/5368 [==============================] - 167s 31ms/step - loss: 0.0368 - accuracy: 0.9850 - val_loss: 0.0012 - val_accuracy: 0.9997
Epoch 2/6
5368/5368 [==============================] - 177s 33ms/step - loss: 0.0060 - accuracy: 0.9980 - val_loss: 0.0081 - val_accuracy: 0.9981
Epoch 3/6
5368/5368 [=====

In [15]:
"""
Load and continue training the 20 GRU model
"""

# Load model from log/model_GRU_20
from keras.models import load_model

model_GRU_20 = load_model('./checkpoints/model_GRU_20') 
model_GRU_20._name = 'model_GRU_20'  # Rename model
model_GRU_20.summary()

Model: "model_GRU_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 20)                2100      
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Continue training
def continue_training(nn_model, epochs=4, batch_size=BATCH_SIZE):
    model_name = str(nn_model.name)
    model_checkpoint_callback = ModelCheckpoint(
        filepath=f'./checkpoints/{model_name}',
        save_weights_only=False,  # Save the whole model
        monitor='val_loss',
        mode='min',
        save_best_only=True)

    log_dir = os.path.join(fp.log_dir, model_name)

    nn_model.summary()
    nn_model.fit(
        train_x,
        train_y,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(val_x, val_y),
        callbacks=[TensorBoard(log_dir), model_checkpoint_callback]
    )

continue_training(model_GRU_20, epochs=6)

Model: "model_GRU_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 20)                2100      
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
5368/5368 [==============================] - 167s 31ms/step - loss: 0.0034 - accuracy: 0.9989 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 2/6
5368/5368 [==============================] - 167s 31ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 5.5850e-04 - val_accuracy: 0.9998
Epoch 3/6
5368/5368 [==============================] - 166s 31ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 3.6128e-04 - val_accuracy: 0.9999
Ep